In [1]:
pip install cplex

     |████████████████████████████████| 39.6 MB 67.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install docplex

     |████████████████████████████████| 610 kB 2.8 MB/s eta 0:00:01
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662819 sha256=43925313c8d3d087c6250adb637c5d44fd4958f97839c22a8c17689c0dcf62f4
  Stored in directory: /Users/alishakhan/Library/Caches/pip/wheels/5e/4c/27/b3b3229ab9a25a089d5c01551d7c2e7efa41462635066784a0
Successfully built docplex
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
from docplex.cp.model import *

In [5]:
model=CpoModel()

In [6]:
import pandas as pd

In [12]:
classes_to_take=[]
val="class"
val=input("Enter the course you would like to take ")
while val.lower()!="exit":
    classes_to_take.append(val)
    val=input("Enter the course you would like to take ")
    

Enter the course you would like to take CS100
Enter the course you would like to take CS115
Enter the course you would like to take EXOT
Enter the course you would like to take EXIT


In [13]:
classes_to_take

['CS100', 'CS115', 'EXOT']

In [ ]:
#constraint: for classes in classes_to_take, the value must be 1
